# Make Results Tables

This notebook generates full HTML tables of results for usage within the online documentation.

In [1]:
from plot_funcs import (get_results, get_ranks_sizes, get_across_ranks, clean_name,
                        get_intra_pipeline_df, clean_model_names, get_highest_performing_df)
from funcs import save_results_table, clean_col_names, save_table
from IPython.display import display

/home/sage/anaconda3/envs/bpt/lib/python3.9/site-packages/nilearn/datasets/__init__.py:93: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  warn("Fetchers from the nilearn.datasets module will be "


In [2]:
results = get_results('../exp/results')
static = {'random': True, 'base': True, 'fs': True, 'ico': True}

Found: 87390 Incomplete: 0


## Base Expiriment - By Parcellation

In [3]:
# Get base
r_df = get_ranks_sizes(results, log=False, 
                       keep_full_name=True,
                       add_raw=True, **static)

# Add median rank
r_df['Median_Rank'] = get_ranks_sizes(
    results, log=False, rank_type='Median_Rank', **static)['Median_Rank']

# Show top result
display(r_df.sort_values('Mean_Rank').iloc[[0]])

# Show the two freesurfer
display(r_df[r_df['full_name'] == 'freesurfer destr'])
display(r_df[r_df['full_name'] == 'freesurfer desikan'])

# Save raw results
save_results_table(r_df, name='raw_results1')

,Parcellation_Type,Mean_Rank,Size,r2,roc_auc,full_name,Median_Rank
194,Existing,46.703704,949,0.081132,0.625744,difumo 1024,34.666667


,Parcellation_Type,Mean_Rank,Size,r2,roc_auc,full_name,Median_Rank
11,Freesurfer Extracted,97.422222,150,0.067749,0.614584,freesurfer destr,100.666667


,Parcellation_Type,Mean_Rank,Size,r2,roc_auc,full_name,Median_Rank
10,Freesurfer Extracted,142.562963,68,0.054734,0.601322,freesurfer desikan,152.666667


## Base Expiriment - By Pipeline

### Intra-Pipeline

In [4]:
# Get df
intra_pipe_df = get_intra_pipeline_df(results, log=False, keep_full_name=True, add_raw=True,
                                      threshold=False, **static)

# Get display col names and change order
intra_pipe_df = clean_col_names(intra_pipe_df)
intra_pipe_df = intra_pipe_df[['Parcellation', 'Pipeline', 'Mean Rank',
                               'Size', 'Mean R2', 'Mean ROC AUC']]

# Save
save_table(intra_pipe_df.sort_values(['Pipeline', 'Mean Rank']),
           'intra_pipe_table')

### Inter-Pipeline

In [8]:
# Get df
inter_pipe_df = clean_model_names(
    get_across_ranks(results, log=False,
                     **static))

# Prep
inter_pipe_df = inter_pipe_df.drop('Parcellation', axis=1)
inter_pipe_df = clean_col_names(inter_pipe_df)
inter_pipe_df = inter_pipe_df[['Parcellation', 'Pipeline', 'Mean Rank', 'Size']]

# Save
save_table(inter_pipe_df.sort_values('Mean Rank'), 'inter_pipe_table')

## Base Expiriment - By Target

In [11]:
# Get data
r_df = get_ranks_sizes(results, avg_targets=False,
                       keep_full_name=True,
                       log=False, threshold=False, **static)

# Get display col names and change order
r_df = clean_col_names(r_df)
r_df = r_df[['Target', 'Parcellation', 'Mean Rank', 'Mean Score', 'Size']]

# Save the table
save_table(r_df.sort_values('Target'), 'by_target_full_results')

## Full Results - By Target

In [15]:
# Get data
r_df = get_ranks_sizes(results, avg_targets=False,
                       keep_full_name=True,
                       log=False, threshold=False,
                       stacked=True, voted=True, grid=True,
                       **static)

# Get display col names and change order
r_df = clean_col_names(r_df)
r_df = r_df[['Target', 'Parcellation', 'Mean Rank', 'Mean Score', 'Size']]

# Save the table
save_table(r_df.sort_values('Target'), 'all_by_target_full_results')

## Full Results - Single + Multiple Parcellations

In [ ]:
r_df = get_ranks_sizes(results, log=False,
                       keep_full_name=True, add_raw=True,
                       stacked=True, voted=True, grid=True,
                       **static)

# Show top result
display(r_df.sort_values('Mean_Rank').iloc[[0]])

# Save raw results
save_results_table(r_df, name='raw_full_results')

## Ensemble Only - By Pipeline

### Intra-Pipeline

In [ ]:
# Get df
intra_pipe_df = get_intra_pipeline_df(results, log=False,
                                      keep_full_name=True,
                                      add_raw=True, stacked=True, voted=True)

# Get display col names and change order
intra_pipe_df = clean_col_names(intra_pipe_df)
intra_pipe_df = intra_pipe_df[['Parcellation', 'Pipeline', 'Mean Rank',
                               'Size', 'Mean R2', 'Mean ROC AUC']]

# Save
save_table(intra_pipe_df.sort_values(['Pipeline', 'Mean Rank']),
           'ensemble_intra_pipe_table')

### Inter-Pipeline

In [ ]:
# Get df
inter_pipe_df = clean_model_names(
    get_across_ranks(results, log=False, stacked=True,
                     voted=True, models=['svm', 'lgbm', 'elastic']))

# Prep
inter_pipe_df = inter_pipe_df.drop('Parcellation', axis=1)
inter_pipe_df = clean_col_names(inter_pipe_df)
inter_pipe_df = inter_pipe_df[['Parcellation', 'Pipeline', 'Mean Rank', 'Size']]

# Save
save_table(inter_pipe_df.sort_values('Mean Rank'), 'ensemble_inter_pipe_table')

## Highest Performing Only

In [ ]:
# Get scores df
r_df = get_highest_performing_df(results).reset_index()

# Prep for display
r_df = clean_model_names(clean_col_names(r_df))
r_df = r_df[['Parcellation', 'Pipeline', 'Mean Rank', 'Size', 'Mean R2', 'Mean ROC AUC']]
r_df['Parcellation'] = r_df['Parcellation'].apply(clean_name)

# Save the table
save_table(r_df.sort_values('Mean Rank'), 'best_of_full_results')

## Highest Performing w/ Extra Special

In [ ]:
# Get scores df
r_df = get_highest_performing_df(results, add_special=True).reset_index()

# Prep for display
r_df = clean_model_names(clean_col_names(r_df))
r_df = r_df[['Parcellation', 'Pipeline', 'Mean Rank', 'Size', 'Mean R2', 'Mean ROC AUC']]
r_df['Parcellation'] = r_df['Parcellation'].apply(clean_name)

# Save the table
save_table(r_df.sort_values('Mean Rank'), 'best_of_full_results_extra')